In [1]:
!pip install scann

     |████████████████████████████████| 11.7MB 4.2MB/s 
     |████████████████████████████████| 320.4MB 50kB/s 
     |████████████████████████████████| 460kB 36.2MB/s 
ERROR: tensorflow 2.3.1 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


In [2]:
import numpy as np
import pandas as pd
import time

import scann

In [3]:
train = pd.read_csv("/content/data/SpotifyFeatures.csv")
train_df = train

col_names = ['acousticness',
               'danceability',
               'energy',
               'instrumentalness',
               'liveness',
               'loudness',
               'speechiness',
               'tempo',
               'valence',
               'popularity']
train = train.drop_duplicates(col_names, keep='first')
train = train[col_names]

print(train.shape[0], train.shape[1])

train_arr = train.to_numpy()

190959 10


In [4]:
normalized_dataset = train_arr / np.linalg.norm(train_arr, axis=1)[:, np.newaxis]
# configure ScaNN as a tree - asymmetric hash hybrid with reordering
# anisotropic quantization as described in the paper; see README

# use scann.scann_ops.build() to instead create a TensorFlow-compatible searcher
searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [16]:
selected_song_index = 110961
query_arr = train_arr[selected_song_index]

# we have been exclusively calling batch search so far; the single-query call has the same API
start = time.time()
neighbors, distances = searcher.search(query_arr, final_num_neighbors=5)
end = time.time()


selected_song = [train_df.to_numpy()[selected_song_index][2], train_df.to_numpy()[selected_song_index][0]]
songs=[]
song_vector = [0,0,0,0,0,0,0,0,0,0]
for index in neighbors:
    song = train_df.to_numpy()[index]
    songs_data = train_df[(train_df['artist_name'] == song[1]) & (train_df['track_name'] == song[2])]
    song_info = [[song[1],song[2]]]
    recommended_song_vector = [song[4], song[5], song[6],
                        song[8], song[9], song[11], 
                        song[12], song[14], song[15],
                        song[17]]
    song_vector = [song_vector[i] + recommended_song_vector[i] for i in range(len(song_vector))]
    for index, row in songs_data.iterrows():
      song_info.append(row['genre']) 
    songs.append(song_info)

song_vector = [song_vector[i]/5 for i in range(len(song_vector))]
    
print(selected_song)
print(songs)

print("Latency (ms):", 1000*(end - start))

print(song_vector/np.linalg.norm(song_vector))


['Amazing', 'Pop']
[[['Kanye West', 'Amazing'], 'Pop', 'Rap'], [['Flo Rida', 'Whistle'], 'Dance', 'Hip-Hop', 'Pop', 'Rap'], [['Machine Gun Kelly', 'LIVEFASTDIEYOUNG'], 'Hip-Hop', 'Rap'], [['Ellie Goulding', 'On My Mind'], 'R&B', 'Dance', 'Pop'], [['Adolphe Adam', 'Giselle / Act 1: No. 8a Final'], 'Opera']]
Latency (ms): 0.40268898010253906
[ 4.26406318e-01  2.71645907e-03  4.80334176e-03  4.47444453e-03
  4.89174159e-04  1.02431577e-03 -7.68456298e-02  5.50252271e-04
  9.01225430e-01  3.61410708e-03]
